In [27]:
# Import login values from config
from configparser import *

In [28]:
# Import packages needed
from snowflake.snowpark import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import StringType, ArrayType
from snowflake.snowpark import Window
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

In [29]:
import sys, json
with open("connection_parameters_true.json") as jsonfile:
    credentials_dict = json.load(jsonfile)

In [30]:
def snowpark_session_create():
    session = Session.builder.configs(credentials_dict).create()
    return session

In [31]:
# Creation of session using the above function
demo_session = snowpark_session_create()

In [32]:
# TODO: Replace with values relevant to you
demo_session.use_database("snowpark")
demo_session.use_schema("gold")

## Pandas

In [33]:
# Creation of a Pandas DataFrame
data = {
    'id' : [1,2,3,4,5],
    'website' : [
        'https://en.wikipedia.org/wiki/Snowflake_Inc',
        'https://en.wikipedia.org/wiki/Amazon_Web_Services',
        'https://en.wikipedia.org/wiki/Google_Cloud_Platform',
        'https://en.wikipedia.org/wiki/Microsoft_Azure',
        'https://en.wikipedia.org/wiki/Apache_Spark',
    ],
    'html' : []
}

for item in data['website']:
    response = urlopen(item)
    html_content = response.read().decode('utf-8')
    #data['html'].append(request.urlopen(item).read())
    data['html'].append(html_content)

pdDataFrame = pd.DataFrame.from_dict(data)

In [34]:
# See Pandas DataFrame
pdDataFrame

,id,website,html
0,1,https://en.wikipedia.org/wiki/Snowflake_Inc,"<!DOCTYPE html>\n<html class=""client-nojs vect..."
1,2,https://en.wikipedia.org/wiki/Amazon_Web_Services,"<!DOCTYPE html>\n<html class=""client-nojs vect..."
2,3,https://en.wikipedia.org/wiki/Google_Cloud_Pla...,"<!DOCTYPE html>\n<html class=""client-nojs vect..."
3,4,https://en.wikipedia.org/wiki/Microsoft_Azure,"<!DOCTYPE html>\n<html class=""client-nojs vect..."
4,5,https://en.wikipedia.org/wiki/Apache_Spark,"<!DOCTYPE html>\n<html class=""client-nojs vect..."


In [35]:
# TODO: Run the below code to create the table
demo_session.sql(
    """
    create or replace table wikipedia_pages (
    id int,
    website string,
    html string
    )

    """
).collect()

[Row(status='Table WIKIPEDIA_PAGES successfully created.')]

In [36]:
# TODO: Create a dataframe that writes the pandas dataframe to a table in Snowflake
spDataFrame = demo_session.write_pandas(
    df = pdDataFrame,
    table_name="wikipedia_pages",
    database= "snowpark",
    schema="gold",
    quote_identifiers=False,
    overwrite=True
)

In [37]:
# TODO: Show that table
spDataFrame.show()

------------------------------------------------------------------------------------------------------------------
|"ID"  |"WEBSITE"                                           |"HTML"                                              |
------------------------------------------------------------------------------------------------------------------
|1     |https://en.wikipedia.org/wiki/Snowflake_Inc         |<!DOCTYPE html>                                     |
|      |                                                    |<html class="client-nojs vector-feature-languag...  |
|      |                                                    |<head>                                              |
|      |                                                    |<meta charset="UTF-8">                              |
|      |                                                    |<title>Snowflake Inc. - Wikipedia</title>           |
|      |                                                    |<script>(function()

In [12]:
# TODO: Create a pandas dataframe called `backtopandas` from the Snowpark dataframe you just created
backtopandas = spDataFrame.toPandas()
# TODO: Show the pandas data frame, just to be sure
backtopandas

,ID,WEBSITE,HTML
0,1,https://en.wikipedia.org/wiki/Snowflake_Inc,"b'<!DOCTYPE html>\n<html class=""client-nojs ve..."
1,2,https://en.wikipedia.org/wiki/Amazon_Web_Services,"b'<!DOCTYPE html>\n<html class=""client-nojs ve..."
2,3,https://en.wikipedia.org/wiki/Google_Cloud_Pla...,"b'<!DOCTYPE html>\n<html class=""client-nojs ve..."
3,4,https://en.wikipedia.org/wiki/Microsoft_Azure,"b'<!DOCTYPE html>\n<html class=""client-nojs ve..."
4,5,https://en.wikipedia.org/wiki/Apache_Spark,"b'<!DOCTYPE html>\n<html class=""client-nojs ve..."


## UDF

Testing

In [13]:
# Function to get rid of HTML tags in a HTML string and remove groups of new lines when they are bundled together
def clean_html_dev(html):
    clean = BeautifulSoup(html, 'lxml').text
    clean = re.sub(r'\n+','\n', clean)
    return clean

In [14]:
# Extract some sample HTML from the pandas dataframe
test_html = backtopandas.loc[0,'HTML']
# Print the result to check everything is working
print(clean_html_dev(test_html))


Snowflake Inc. - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
		Navigation
	
Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate
		Contribute
	
HelpLearn to editCommunity portalRecent changesUpload file
Languages
Language links are at the top of the page across from the title.
Search
Search
Create accountLog in
Personal tools
 Create account Log in
		Pages for logged out editors learn more
ContributionsTalk
Contents
move to sidebar
hide
(Top)
1History
2References
3External links
Toggle the table of contents
Toggle the table of contents
Snowflake Inc.
7 languages
DeutschEspañolفارسیFrançaisעבריתNorsk bokmålSvenska
Edit links
ArticleTalk
English
ReadEditView history
Tools
Tools
move to sidebar
hide
		Actions
	
ReadEditView history
		General
	
What links hereRelated changesUpload fileSpecial pagesPermanent linkPage informationCite this pageWikidata item
		Print/export
	
Download as PDFPrintable version
		In other projects
	
Wikimedia Commons

## Developing

In [21]:
# TODO: Register a UDF with Snowpark based on the `clean_html_dev` function above
@udf(return_type=StringType(), input_types=[StringType()], name="clean_html",
     packages =["BeautifulSoup4","lxml"], replace=True)
def clean_html(html):
    clean = BeautifulSoup(html, 'lxml').text
    clean = re.sub(r'\n+','\n', clean)
    return clean

The version of package 'lxml' in the local environment is 4.9.3, which does not fit the criteria for the requirement 'lxml'. Your UDF might not work when the package version is different between the server and your local environment.


In [22]:
# Create a new column that applies the UDF to the `HTML`` column called `website_text`, and then drop `HTML`
ndf = spDataFrame.withColumn(
    "website_text",
    clean_html("html")
).drop("html")

In [38]:
# Show the resulting data frame to check it works
ndf.show()

------------------------------------------------------------------------------------------------------------------
|"ID"  |"WEBSITE"                                           |"WEBSITE_TEXT"                                      |
------------------------------------------------------------------------------------------------------------------
|1     |https://en.wikipedia.org/wiki/Snowflake_Inc         |                                                    |
|      |                                                    |Snowflake Inc. - Wikipedia                          |
|      |                                                    |Jump to content                                     |
|      |                                                    |Main menu                                           |
|      |                                                    |Main menu                                           |
|      |                                                    |move to sidebar    

## Flattening

In [39]:
# TODO: Run the below SQL
demo_session.sql(
    """ 
    create or replace stage demo_stg
        url = 's3://frostyfridaychallenges/challenge_25/'
        file_format = (type = json);
    """
).collect()
demo_session.sql(
    """
    create or replace table demo_json as
        select $1 as data
        from @demo_stg (pattern => '.*ber_7d.*');
    """
).collect()

[Row(status='Table DEMO_JSON successfully created.')]

In [40]:
# TODO: Run the below code
json_df = demo_session.table("demo_json")

In [41]:
# TODO: .show() the result
json_df.show()

------------------------------------------------------
|"DATA"                                              |
------------------------------------------------------
|{                                                   |
|  "sources": [                                      |
|    {                                               |
|      "distance": 2521,                             |
|      "dwd_station_id": "13696",                    |
|      "first_record": "2010-01-01T00:00:00+00:00",  |
|      "height": 60,                                 |
|      "id": 46604,                                  |
|      "last_record": "2022-11-20T23:00:00+00:00",   |
|      "lat": 51.5966,                               |
|      "lon": 7.4048,                                |
|      "observation_type": "historical",             |
|      "station_name": "Waltrop-Abdinghof",          |
|      "wmo_station_id": "H443"                      |
|    },                                              |
|    {    

In [42]:
# TODO: Step 1 = Flatten the sources column
step1 = json_df.join_table_function(
    "flatten",
    col("data"),
    lit("sources")
).select("index","path","value")

In [43]:
# TODO: Show the result of Step 1 to check it worked
step1.show()

-------------------------------------------------------------------------
|"INDEX"  |"PATH"      |"VALUE"                                         |
-------------------------------------------------------------------------
|0        |sources[0]  |{                                               |
|         |            |  "distance": 2521,                             |
|         |            |  "dwd_station_id": "13696",                    |
|         |            |  "first_record": "2010-01-01T00:00:00+00:00",  |
|         |            |  "height": 60,                                 |
|         |            |  "id": 46604,                                  |
|         |            |  "last_record": "2022-11-20T23:00:00+00:00",   |
|         |            |  "lat": 51.5966,                               |
|         |            |  "lon": 7.4048,                                |
|         |            |  "observation_type": "historical",             |
|         |            |  "station_nam

In [45]:
# TODO: Step 2, extract each column
step2 = (
    step1.withColumn(
        "distance",
        json_extract_path_text("value",lit("distance"))
    )
    .withColumn(
        "dwd_station_id",
        json_extract_path_text("value",lit("dwd_station_id"))
    )
    .withColumn(
        "first_record",
        json_extract_path_text("value",lit("first_record"))
    )
    .withColumn(
        "height",
        json_extract_path_text("value",lit("height"))
    )
    .withColumn(
        "id",
        json_extract_path_text("value",lit("id"))
    )
    .withColumn(
        "last_record",
        json_extract_path_text("value",lit("last_record"))
    )
    .withColumn(
        "lat",
        json_extract_path_text("value",lit("lat"))
    )
    .withColumn(
        "lon",
        json_extract_path_text("value",lit("lon"))
    )
    .withColumn(
        "observation_type",
        json_extract_path_text("value",lit("observation_type"))
    )
    .withColumn(
        "station_name",
        json_extract_path_text("value",lit("station_name"))
    )
    .withColumn(
        "wmo_station_id",
        json_extract_path_text("value",lit("wmo_station_id"))
    )
).drop("value")

In [46]:
# TODO: Show the results to check
step2.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"INDEX"  |"PATH"      |"DISTANCE"  |"DWD_STATION_ID"  |"FIRST_RECORD"             |"HEIGHT"  |"ID"   |"LAST_RECORD"              |"LAT"    |"LON"   |"OBSERVATION_TYPE"  |"STATION_NAME"        |"WMO_STATION_ID"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0        |sources[0]  |2521        |13696             |2010-01-01T00:00:00+00:00  |60        |46604  |2022-11-20T23:00:00+00:00  |51.5966  |7.4048  |historical          |Waltrop-Abdinghof     |H443              |
|1        |sources[1]  |14187       |03795             |2010-01-01T00:00:00+00:00  |48.5      |8228   |2022-11-20T23:00:00+00:00  |51.7066  |7.3

## Window Function

In [47]:
# Create data frames for `catalog_sales`, `warehouse` and `date_dim` tables from the `tpcds_sf10tcl` schema
catalog_sales_df = (
    demo_session.table("snowflake_sample_data.tpcds_sf10tcl.catalog_sales")
    .select("cs_order_number", "cs_ext_tax", "cs_warehouse_sk", "cs_sold_date_sk")
)
warehouse_df = (
    demo_session.table("snowflake_sample_data.tpcds_sf10tcl.warehouse")
    .select("w_warehouse_name", "w_state", "w_warehouse_sk")
)
date_dim_df = (
    demo_session.table("snowflake_sample_data.tpcds_sf10tcl.date_dim")
    .select("d_date", "d_date_sk")
    .filter((date_part("YEAR", "d_date") == 2001) & (date_part("MONTH", "d_date") == 3)) # This one is filtered to the required month-year
)
# Join the three data frames together, get rid of surrogate key columns and rename kept columns
catalog_df = (
    catalog_sales_df.join(
        warehouse_df,
        catalog_sales_df.cs_warehouse_sk == warehouse_df.w_warehouse_sk
    )
    .join(
        date_dim_df,
        catalog_sales_df.cs_sold_date_sk == date_dim_df.d_date_sk
    )
    .select("cs_order_number", "d_date", "w_warehouse_name", "w_state", "cs_ext_tax")
    .to_df("order_number", "date", "warehouse_name", "state", "ext_tax")
)

In [48]:
catalog_df.show()

----------------------------------------------------------------------------
|"ORDER_NUMBER"  |"DATE"      |"WAREHOUSE_NAME"      |"STATE"  |"EXT_TAX"  |
----------------------------------------------------------------------------
|991787670       |2001-03-12  |New, net purposes     |LA       |18.98      |
|991642555       |2001-03-12  |New, net purposes     |LA       |10.67      |
|991471048       |2001-03-12  |Other, alon           |CO       |586.16     |
|991947222       |2001-03-12  |Rooms cook            |TN       |60.27      |
|991706568       |2001-03-12  |Local, mass universi  |WV       |389.78     |
|991935157       |2001-03-12  |Months woul           |NM       |29.17      |
|991556585       |2001-03-12  |Months woul           |NM       |14.13      |
|991933739       |2001-03-12  |Soon regular workers  |IN       |17.65      |
|991629629       |2001-03-12  |Important dem         |NC       |2.47       |
|991935965       |2001-03-12  |Important dem         |NC       |4.19       |

In [49]:
# Aggregate the data frame
catalog_df_agg = (
    catalog_df.group_by("state","warehouse_name")
    .agg(median("ext_tax").alias("median_ext_tax"))
)

In [51]:
# Show the result to check this has worked
catalog_df_agg.order_by("median_ext_tax").show()

-----------------------------------------------------
|"STATE"  |"WAREHOUSE_NAME"      |"MEDIAN_EXT_TAX"  |
-----------------------------------------------------
|FL       |Surprising emot       |34.31000          |
|IN       |Soon regular workers  |34.32000          |
|LA       |Conventional childr   |34.35000          |
|NM       |Months woul           |34.35000          |
|LA       |New, net purposes     |34.36000          |
|NE       |Bad cards must make.  |34.36000          |
|MI       |Plain, reluctant      |34.36000          |
|NY       |Agricultural measur   |34.37000          |
|NC       |Important dem         |34.37000          |
|TN       |Rooms cook            |34.37000          |
-----------------------------------------------------



In [52]:
# TODO: Define the window, partitioned by state and ordered by that `median_ext_tax` column
window = Window.partition_by("state").order_by("median_ext_tax")

In [53]:
# TODO: Rank the dataframe according to the window, call this `ranked_df`
ranked_df = catalog_df_agg.select("state","warehouse_name",rank().over(window).as_("rank_by_state"))

In [54]:
# Show the result
ranked_df.order_by("state","rank_by_state").show(30)

----------------------------------------------------
|"STATE"  |"WAREHOUSE_NAME"      |"RANK_BY_STATE"  |
----------------------------------------------------
|CO       |Total sports leap a   |1                |
|CO       |Other, alon           |1                |
|CO       |Please central m      |3                |
|CO       |NULL                  |4                |
|CO       |Governments should    |5                |
|FL       |Surprising emot       |1                |
|GA       |New proceedings fo    |1                |
|IN       |Soon regular workers  |1                |
|LA       |Conventional childr   |1                |
|LA       |New, net purposes     |2                |
|MI       |Plain, reluctant      |1                |
|MO       |Therefore urg         |1                |
|MO       |Empty, middle women   |2                |
|NC       |Important dem         |1                |
|NC       |Unexpected sup        |2                |
|NE       |Bad cards must make.  |1           

In [ ]:
#demo_session.close()